# StatsDataReader - e-Stat統計データ取得ガイド

このノートブックでは、jpy-datareaderライブラリの`StatsDataReader`を使用して、日本の政府統計e-Statから実際の統計データを取得する方法を学習します。

## 概要

`StatsDataReader`は、e-Stat APIの「統計データ取得」機能を使用して、以下のデータを取得できます：
- 実際の統計数値データ
- 各種分類コードに対応する名称情報
- 自動的なページネーション処理（10万件超のデータ対応）
- 条件指定による絞り込み検索
- 単位別でのデータ分割機能

## 事前準備

1. **統計表IDの特定**
   - `StatsListReader`で統計表IDを特定
   - 必要に応じて`MetaInfoReader`で分類構造を把握

2. **必要なライブラリ**
   - `jpy-datareader`
   - `pandas`
   - `python-dotenv` (環境変数管理用)

## 1. ライブラリのインポートと設定

In [ ]:
import os
import sys
from pathlib import Path
import requests
import pandas as pd
from dotenv import load_dotenv

# 日本語表示の設定
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', 40)
pd.set_option('display.float_format', '{:.2f}'.format)

## 2. StatsDataReaderインポート

In [ ]:
from jpy_datareader.estat import StatsDataReader, colname_to_japanese

## 3. APIキーの読み込み

In [ ]:
# APIキーの読み込み
load_dotenv()
appId = os.getenv("ESTAT_APP_ID")

if not appId:
    print("⚠️ APIキーが設定されていません")
    print("以下の環境変数のいずれかを設定してください:")
    print("- ESTAT_APP_ID")
    print("- E_STAT_APPLICATION_ID")
    print("- ESTAT_APPLICATION_ID")
    # 直接設定する場合（テスト用）
    # appId = "あなたのAPIキーをここに入力"
else:
    print(f"✅ APIキーが正常に読み込まれました: {appId[:10]}...")

## 基本的な使用方法

### 4. 統計表IDの設定と基本的なデータ取得

In [ ]:
# 国勢調査の統計表IDを使用（例）
statsDataId = "0002070010"  # 国勢調査：男女別人口及び世帯の種類別世帯数

print(f"=== 統計表ID: {statsDataId} の基本データ取得 ===")

# StatsDataReaderインスタンスの作成（小さめのlimitで試す）
statsdata = StatsDataReader(
    api_key=appId, 
    statsDataId=statsDataId, 
    limit=1000  # 最初は1000件に制限
)

print("StatsDataReaderインスタンスを作成しました")
print(f"統計表ID: {statsdata.statsDataId}")
print(f"データ取得制限: {statsdata.limit}件")

### 5. メタ情報の事前取得

In [ ]:
# メタ情報を取得してデータ規模を確認
print("=== メタ情報の取得 ===")
metadata = statsdata.get_metadata()

print(f"統計名: {statsdata.STAT_NAME}")
print(f"表題: {statsdata.TITLE}")
print(f"調査年月: {statsdata.SURVEY_DATE}")
print(f"総データ件数: {statsdata.OVERALL_TOTAL_NUMBER:,}件")

# データ規模の確認
total_records = statsdata._get_total_number()
print(f"\n=== データ規模の確認 ===")
print(f"総レコード数: {total_records:,}件")

if total_records > 100000:
    print("⚠️ データが10万件を超えているため、自動ページネーション処理が実行されます")
else:
    print("✅ データは10万件以下のため、一括取得が可能です")

### 6. JSON形式での生データ確認

In [ ]:
# JSON形式でAPI生レスポンスを確認
print("=== JSON形式での生データ確認 ===")
json_data = statsdata.read_json()

# APIレスポンスの構造確認
get_stats_data = json_data.get('GET_STATS_DATA', {})
print("APIレスポンスの主要キー:")
for key in get_stats_data.keys():
    print(f"- {key}")

# 統計データの基本情報確認
statistical_data = get_stats_data.get('STATISTICAL_DATA', {})
result_inf = statistical_data.get('RESULT_INF', {})

print(f"\n=== 今回取得したデータ情報 ===")
print(f"総件数: {result_inf.get('TOTAL_NUMBER')}件")
print(f"取得範囲: {result_inf.get('FROM_NUMBER')} - {result_inf.get('TO_NUMBER')}")

# メタデータをインスタンス属性として保存
statsdata._store_params_in_attrs(json_data)
print(f"統計表タイトル: {statsdata.TITLE}")
print(f"データ名: {statsdata.StatsDataName}")

## データ処理のステップ別実行

### 7. VALUE データの基本変換

In [ ]:
# ステップ1: JSON の VALUE データを DataFrame に変換
print("=== ステップ1: VALUE データの DataFrame 変換 ===")
value_df = statsdata._statsjson_to_dataframe(json_data)

print(f"変換後のデータ形状: {value_df.shape}")
print(f"カラム: {list(value_df.columns)}")
print(f"カテゴリカラム: {statsdata.category_columns}")

print(f"\n=== VALUE データのサンプル ===")
print(value_df.head())

# 値の分布確認
print(f"\n=== 値の分布（上位10種類）===")
value_counts = value_df['value'].value_counts().head(10)
for value, count in value_counts.items():
    print(f"  {value}: {count}件")

### 8. 欠損値の処理

In [ ]:
print("=== ステップ2: 欠損値の処理 ===")
value_df2 = statsdata._handle_missing_values(value_df, json_data)

print(f"欠損値処理後のデータ形状: {value_df2.shape}")

# 欠損値処理前後の比較
print(f"\n=== 欠損値処理の効果 ===")
print("処理前の値分布（上位5種類）:")
original_counts = value_df['value'].value_counts(dropna=False).head()
for value, count in original_counts.items():
    print(f"  {value}: {count}件")

print("\n処理後の値分布（上位5種類）:")
processed_counts = value_df2['value'].value_counts(dropna=False).head()
for value, count in processed_counts.items():
    print(f"  {value}: {count}件")

# データ型の確認
print(f"\n処理後のvalueカラムのデータ型: {value_df2['value'].dtype}")

### 9. 分類メタデータとの結合

In [ ]:
# ステップ3: 分類メタデータとの結合
print("=== ステップ3: 分類メタデータとの結合 ===")
value_df3 = statsdata._merge_class_metadata(value_df2, json_data)

print(f"結合後のデータ形状: {value_df3.shape}")
print(f"結合後のカラム数: {len(value_df3.columns)}")

print(f"\n=== 結合後のカラム一覧 ===")
for i, col in enumerate(value_df3.columns):
    print(f"{i+1:2d}. {col}")

print(f"\n=== 結合後のデータサンプル ===")
# 主要なカラムのみを表示
display_cols = [col for col in value_df3.columns if any(keyword in col.lower() 
                for keyword in ['code', 'name', 'value']) and not col.endswith('level')][:8]
if display_cols:
    print(value_df3[display_cols].head())

### 10. カラム名の日本語化

In [ ]:
# ステップ4: カラム名の日本語変換
print("=== ステップ4: カラム名の日本語変換 ===")
value_df4 = statsdata._apply_colname_transformations(value_df3, json_data)

print(f"日本語化後のデータ形状: {value_df4.shape}")

print(f"\n=== 日本語化後のカラム一覧 ===")
for i, col in enumerate(value_df4.columns):
    print(f"{i+1:2d}. {col}")

print(f"\n=== クラス名マッピング ===")
if hasattr(statsdata, 'CLASS_NAME_MAPPING'):
    for class_id, class_name in statsdata.CLASS_NAME_MAPPING.items():
        print(f"  {class_id} → {class_name}")

print(f"\n表章項目カラム名: {statsdata.tab_colname}")

print(f"\n=== 日本語化後のデータサンプル ===")
print(value_df4.head())

In [ ]:
statsdata.en_colnames

In [ ]:
statsdata.jp_colnames

In [ ]:
statsdata.EN_JP_MAPPING

## 実践的なデータ取得

### 11. 完全なデータ取得

In [ ]:

# 完全なデータ取得プロセス
print("=== 完全なデータ取得プロセス ===")

# より大きなデータセットで試す（ページネーション機能を含む）
statsdata_full = StatsDataReader(
    api_key=appId, 
    statsDataId=statsDataId, 
    limit=100500  # 10万件を少し超える設定
)

# データ取得
df_full = statsdata_full.read()

print(f"取得完了")
print(f"最終データ形状: {df_full.shape}")
print(f"カラム数: {len(df_full.columns)}")
print(f"総データ件数: {statsdata_full.OVERALL_TOTAL_NUMBER:,}件")

# ページネーション情報の確認
if hasattr(statsdata_full, 'FROM_NUMBER') and hasattr(statsdata_full, 'TO_NUMBER'):
    print(f"取得範囲: {statsdata_full.FROM_NUMBER:,} - {statsdata_full.TO_NUMBER:,}")

# 重複チェック
duplicates = df_full[df_full.duplicated()]
print(f"重複レコード数: {len(duplicates)}件")

print(f"\n=== 最終データのサンプル（100,000件目周辺）===")
if len(df_full) > 100005:
    sample_data = df_full.iloc[100000:100005]
    print(sample_data.to_string(index=False))

### 12. 単位別データ分割

In [ ]:
# 単位別でのデータ分割
print("=== 単位別データ分割 ===")

# 単位別に分割
datasets_by_unit = statsdata_full._split_by_units(df_full)

print(f"分割されたデータセット数: {len(datasets_by_unit)}")

print(f"\n=== 各単位のデータセット概要 ===")
for unit, unit_df in datasets_by_unit.items():
    print(f"単位: {unit}")
    print(f"  データ件数: {len(unit_df):,}件")
    print(f"  データ形状: {unit_df.shape}")
    
    # 値の統計情報
    if '値' in unit_df.columns:
        values = unit_df['値'].dropna()
        if len(values) > 0:
            print(f"  値の範囲: {values.min():.0f} - {values.max():.0f}")
            print(f"  平均値: {values.mean():.2f}")
    print()

## 高度な検索・絞り込み機能

### 13. 条件指定によるデータ絞り込み

In [ ]:
# 特定条件でのデータ取得例
print("=== 条件指定によるデータ絞り込み ===")

# まず利用可能な分類コードを確認（MetaInfoReaderから）
from jpy_datareader.estat import MetaInfoReader

metainfo = MetaInfoReader(api_key=appId, statsDataId=statsDataId)
meta_classes = metainfo.read_class_objs()

print("=== 利用可能な分類情報 ===")
filter_examples = {}

for class_data in meta_classes:
    class_id = class_data['id']
    class_name = class_data['name']
    df = class_data['meta_dataframe']
    
    # コードカラムを特定
    code_cols = [col for col in df.columns if 'コード' in col]
    if code_cols and len(df) > 0:
        code_col = code_cols[0]
        sample_codes = df[code_col].head(3).tolist()
        print(f"{class_id} ({class_name}): {sample_codes}")
        
        # フィルタ例の準備
        if class_id in ['area', 'time'] and len(sample_codes) > 0:
            filter_examples[class_id] = sample_codes[0]

print(f"\n=== 絞り込み例の実行 ===")

# 地域絞り込みの例（北海道など）
if 'area' in filter_examples:
    area_code = filter_examples['area']
    print(f"地域コード {area_code} でのデータ取得:")
    
    statsdata_filtered = StatsDataReader(
        api_key=appId,
        statsDataId=statsDataId,
        cdArea=area_code,  # 地域コード指定
        limit=1000
    )
    
    df_area = statsdata_filtered.read()
    print(f"  絞り込み後データ件数: {len(df_area):,}件")
    
    # 地域情報の確認
    area_cols = [col for col in df_area.columns if '地域' in col and 'コード' not in col]
    if area_cols:
        unique_areas = df_area[area_cols[0]].unique()
        print(f"  対象地域: {unique_areas}")

# 時系列絞り込みの例
if 'time' in filter_examples:
    time_code = filter_examples['time']
    print(f"\n時間コード {time_code} でのデータ取得:")
    
    statsdata_time = StatsDataReader(
        api_key=appId,
        statsDataId=statsDataId,
        cdTime=time_code,  # 時間コード指定
        limit=1000
    )
    
    df_time = statsdata_time.read()
    print(f"  絞り込み後データ件数: {len(df_time):,}件")
    
    # 時系列情報の確認
    time_cols = [col for col in df_time.columns if '時間' in col and 'コード' not in col]
    if time_cols:
        unique_times = df_time[time_cols[0]].unique()
        print(f"  対象時期: {unique_times}")

### 14. 複数条件での絞り込み

In [ ]:

# 複数条件での絞り込み例
print("=== 複数条件での絞り込み ===")

# 範囲指定による絞り込み
print("範囲指定による絞り込み例:")

statsdata_range = StatsDataReader(
    api_key=appId,
    statsDataId=statsDataId,
    cdAreaFrom="01000",  # 北海道から
    cdAreaTo="02000",    # 青森県まで
    limit=2000
)

df_range = statsdata_range.read()
print(f"北海道〜青森県のデータ件数: {len(df_range):,}件")

# 地域の確認
if len(df_range) > 0:
    area_cols = [col for col in df_range.columns if '地域' in col and 'コード' not in col]
    if area_cols:
        unique_areas = df_range[area_cols[0]].unique()
        print(f"対象地域: {unique_areas}")

# レベル指定による絞り込み
print(f"\nレベル指定による絞り込み例:")

statsdata_level = StatsDataReader(
    api_key=appId,
    statsDataId=statsDataId,
    lvArea=2,  # 都道府県レベル
    limit=2000
)

df_level = statsdata_level.read()
print(f"都道府県レベルのデータ件数: {len(df_level):,}件")

# レベル情報の確認
if len(df_level) > 0:
    level_cols = [col for col in df_level.columns if '地域' in col and 'レベル' in col]
    area_cols = [col for col in df_level.columns if '地域' in col and 'コード' not in col and 'レベル' not in col]
    
    if level_cols and area_cols:
        level_dist = df_level[level_cols[0]].value_counts()
        print(f"レベル分布: {dict(level_dist)}")
        
        sample_areas = df_level[area_cols[0]].unique()[:5]
        print(f"サンプル地域: {sample_areas}")

## データ分析とエクスポート

### 15. 基本統計とデータ分析

In [ ]:
# データの基本統計と分析
print("=== データの基本統計と分析 ===")

# 最新の完全データを使用
analysis_df = df_full.copy()

print(f"分析対象データ: {analysis_df.shape}")

# 数値データの基本統計
value_col = '値' if '値' in analysis_df.columns else 'value'
if value_col in analysis_df.columns:
    numeric_data = pd.to_numeric(analysis_df[value_col], errors='coerce')
    
    print(f"\n=== 数値データの基本統計 ===")
    print(f"総データ件数: {len(numeric_data):,}件")
    print(f"有効データ件数: {numeric_data.notna().sum():,}件")
    print(f"欠損データ件数: {numeric_data.isna().sum():,}件")
    
    if numeric_data.notna().sum() > 0:
        valid_data = numeric_data.dropna()
        print(f"最小値: {valid_data.min():,.0f}")
        print(f"最大値: {valid_data.max():,.0f}")
        print(f"平均値: {valid_data.mean():,.2f}")
        print(f"中央値: {valid_data.median():,.0f}")

# 分類別の集計
print(f"\n=== 分類別データ集計 ===")

# 表章項目別の集計
if statsdata_full.tab_colname in analysis_df.columns:
    tab_dist = analysis_df[statsdata_full.tab_colname].value_counts()
    print(f"表章項目別件数（上位5項目）:")
    for item, count in tab_dist.head().items():
        print(f"  {item}: {count:,}件")

# 地域別の集計
area_cols = [col for col in analysis_df.columns if '地域' in col and 'コード' not in col and 'レベル' not in col]
if area_cols:
    area_col = area_cols[0]
    area_dist = analysis_df[area_col].value_counts()
    print(f"\n地域別件数（上位5地域）:")
    for area, count in area_dist.head().items():
        print(f"  {area}: {count:,}件")

# 時系列別の集計
time_cols = [col for col in analysis_df.columns if '時間' in col and 'コード' not in col and 'レベル' not in col]
if time_cols:
    time_col = time_cols[0]
    time_dist = analysis_df[time_col].value_counts().sort_index()
    print(f"\n時系列別件数（最新5期間）:")
    for time_period, count in time_dist.tail().items():
        print(f"  {time_period}: {count:,}件")

### 16. データの保存とエクスポート

In [ ]:

# データの保存とエクスポート
print("=== データの保存とエクスポート ===")

# メタデータの保存
metadata_info = {
    'statsDataId': statsDataId,
    'STAT_NAME': statsdata_full.STAT_NAME,
    'TITLE': statsdata_full.TITLE,
    'SURVEY_DATE': statsdata_full.SURVEY_DATE,
    'OVERALL_TOTAL_NUMBER': statsdata_full.OVERALL_TOTAL_NUMBER,
    'data_shape': f"{df_full.shape[0]}x{df_full.shape[1]}",
    'columns_count': len(df_full.columns),
    'export_date': pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S')
}

metadata_df = pd.DataFrame([metadata_info])
metadata_file = f"統計データ情報_{statsDataId}.csv"
metadata_df.to_csv(metadata_file, index=False, encoding='utf-8-sig')
print(f"✅ メタデータを {metadata_file} に保存しました")

# 完全データの保存（サンプル版）
sample_size = min(10000, len(df_full))
sample_df = df_full.head(sample_size).copy()

output_file = f"統計データ_{statsDataId}_サンプル{sample_size}件.csv"
sample_df.to_csv(output_file, index=False, encoding='utf-8-sig')
print(f"✅ サンプルデータ（{sample_size:,}件）を {output_file} に保存しました")

# ファイル情報の表示
file_size = Path(output_file).stat().st_size / 1024 / 1024  # MB
print(f"ファイルサイズ: {file_size:.2f} MB")

# 単位別データの保存
if 'datasets_by_unit' in locals() and len(datasets_by_unit) > 1:
    print(f"\n=== 単位別データの保存 ===")
    for unit, unit_df in datasets_by_unit.items():
        # ファイル名に適さない文字を除去
        safe_unit = "".join(c for c in str(unit) if c.isalnum() or c in (' ', '-', '_')).strip()
        unit_file = f"統計データ_{statsDataId}_{safe_unit}.csv"
        
        # サンプルサイズの調整
        unit_sample_size = min(5000, len(unit_df))
        unit_sample = unit_df.head(unit_sample_size)
        
        unit_sample.to_csv(unit_file, index=False, encoding='utf-8-sig')
        print(f"✅ {unit}単位データ（{unit_sample_size:,}件）を {unit_file} に保存しました")

print(f"\n=== データ取得・処理完了 ===")
print(f"統計表ID: {statsDataId}")
print(f"統計名: {statsdata_full.STAT_NAME}")
print(f"最終データ件数: {len(df_full):,}件")
print(f"処理時刻: {pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S')}")


## まとめ

### StatsDataReaderの主要な機能

1. **基本データ取得**: `read()`メソッドで統計データを取得
2. **自動ページネーション**: 10万件超のデータに対する自動分割処理
3. **条件絞り込み**: 
   - 地域指定: `cdArea`, `lvArea`, `cdAreaFrom`, `cdAreaTo`
   - 時系列指定: `cdTime`, `lvTime`, `cdTimeFrom`, `cdTimeTo`
   - 表章項目指定: `cdTab`, `lvTab`, `cdTabFrom`, `cdTabTo`
   - 分類指定: `cdCat01`, `cdCat02`, `cdCat03`等
4. **データ形式**: DataFrame（`read()`）、JSON（`read_json()`）、単位別分割（`split_by_unit=True`）

### 活用のポイント

- **事前のメタ情報確認**: `MetaInfoReader`でデータ構造を把握してから取得
- **適切な絞り込み**: 必要なデータのみを指定して効率的に取得
- **大容量データ対応**: 自動ページネーション機能を活用した安全な大量データ取得
- **データクリーニング**: 欠損値処理と特殊文字の自動変換

### 次のステップ

取得したデータを活用して、時系列分析、地域比較、項目別分析などの統計分析を実行できます。また、複数の統計表を組み合わせた横断的な分析や、可視化ツールとの連携も可能です。
